In [194]:
import scipy as sp
def decode(dev_data):
    dev_contexts_index = []
    dev_questions = []
    dev_que_id = []
    dev_map = {}
    for i,line in enumerate(dev_data):
        dev_contexts_index.append(line[0])
        dev_questions.append(line[1])
        dev_que_id.append(line[2])
        dev_candidates = []
        for j in range(3,len(line),3):
            dev_candidates.append((line[j],line[j+1],line[j+2]))
        dev_map[i]=dev_candidates
    return dev_contexts_index,dev_questions,dev_que_id,dev_map

# train set has no id issue comes with a new function
def trn_decode(trn_data):
    trn_contexts_index = []
    trn_questions = []
    trn_map = {}
    for i,line in enumerate(trn_data):
        trn_contexts_index.append(line[0])
        trn_questions.append(line[1])
        trn_candidates = []
        for j in range(2,len(line),3):
            trn_candidates.append((line[j],line[j+1],line[j+2]))
        trn_map[i]=trn_candidates
    return trn_contexts_index,trn_questions,trn_map

import csv
dev_data = []
trn_data = []
tst_data = []
with open('dev_data.csv') as Dev_DataFile:
    raw_dev = csv.reader(Dev_DataFile)
    for row in raw_dev:
        dev_data.append(row)

with open('train_data.csv') as Trn_DataFile:
    raw_trn = csv.reader(Trn_DataFile)
    for row in raw_trn:
#         print(len(row))
        trn_data.append(row)

with open('test_data.csv') as Tst_DataFile:
    raw_tst = csv.reader(Tst_DataFile)
    for row in raw_tst:
#         print(len(row))
        tst_data.append(row)

# get all useful list info of each dataset
dev_contexts_index, dev_questions, dev_que_id, dev_map = decode(dev_data)
tst_contexts_index, tst_questions, tst_que_id, tst_map = decode(tst_data)
trn_contexts_index, trn_questions, trn_map = trn_decode(trn_data)

# vocabulary processing and collection of each dataset contexts
from sklearn.feature_extraction.text import TfidfVectorizer  
with open('dev_contexts.txt') as f:
    dev_contexts = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
dev_contexts = [x.strip() for x in dev_contexts] 

with open('train_contexts.txt') as f:
    train_contexts = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
train_contexts = [x.strip() for x in train_contexts]

with open('test_contexts.txt') as f:
    test_contexts = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
test_contexts = [x.strip() for x in test_contexts]

dev_vec = TfidfVectorizer()
dev_text_vec = vec.fit_transform(dev_contexts)
dev_voc = vec.vocabulary_

tst_vec = TfidfVectorizer()
tst_text_vec = vec.fit_transform(test_contexts)
tst_voc = vec.vocabulary_

trn_vec = TfidfVectorizer()
trn_text_vec = vec.fit_transform(train_contexts)
trn_voc = vec.vocabulary_

# Train Matching Word Frequencies
Trn_MWF = []
for i in trn_map.keys():
    for j in trn_map[i]:
        trn_wmf_feq_sum = 0
        for word in j[0].split():
            if word in trn_questions[i].split():
                if word.lower() in trn_voc:
                    index = trn_voc[word.lower()]
                    trn_wmf_feq_sum += trn_text_vec[trn_contexts_index[i],index]
        Trn_MWF.append(trn_wmf_feq_sum)

# Train Span Word Frequencies
Trn_SWF = []
for i in trn_map.keys():
    for j in trn_map[i]:
        trn_swf_feq_sum = 0
        for word in j[0].split():
            if word.lower() in trn_voc:
                index = trn_voc[word.lower()]
                trn_swf_feq_sum += trn_text_vec[trn_contexts_index[i],index]
        Trn_SWF.append(trn_swf_feq_sum)    

# Development Matching Word Frequencies
Dev_MWF = []
for i in dev_map.keys():
    for j in dev_map[i]:
        dev_wmf_feq_sum = 0
        for word in j[0].split():
            if word in dev_questions[i].split():
                if word.lower() in dev_voc:
                    index = dev_voc[word.lower()]
                    dev_wmf_feq_sum += dev_text_vec[dev_contexts_index[i],index]
    Dev_MWF.append(dev_wmf_feq_sum)

# Development Span Word Frequencies
Dev_SWF = []
for i in dev_map.keys():
    for j in dev_map[i]:
        dev_swf_feq_sum = 0
        for word in j[0].split():
            if word.lower() in dev_voc:
                index = dev_voc[word.lower()]
                dev_swf_feq_sum += dev_text_vec[dev_contexts_index[i],index]
        Dev_SWF.append(dev_swf_feq_sum)    

# Test Matching Word Frequencies
Tst_MWF = []
for i in tst_map.keys():
    for j in tst_map[i]:
        tst_wmf_feq_sum = 0
        for word in j[0].split():
            if word in tst_questions[i].split():
                if word.lower() in tst_voc:
                    index = tst_voc[word.lower()]
                    tst_wmf_feq_sum += tst_text_vec[tst_contexts_index[i],index]
        Tst_MWF.append(tst_wmf_feq_sum)

# Test Span Word Frequencies
Tst_SWF = []
for i in tst_map.keys():
    for j in tst_map[i]:
        tst_swf_feq_sum = 0
        for word in j[0].split():
            if word.lower() in tst_voc:
                index = tst_voc[word.lower()]
                tst_swf_feq_sum += tst_text_vec[tst_contexts_index[i],index]
        Tst_SWF.append(tst_swf_feq_sum) 


thefile = open('Trn_MWF.txt', 'w')
for item in Trn_MWF:
    thefile.write("%s\n" % item)
thefile = open('Trn_SWF.txt', 'w')
for item in Trn_SWF:
    thefile.write("%s\n" % item)
thefile = open('Dev_MWF.txt', 'w')
for item in Dev_MWF:
    thefile.write("%s\n" % item)
thefile = open('Dev_SWF.txt', 'w')
for item in Dev_SWF:
    thefile.write("%s\n" % item)
thefile = open('Tst_MWF.txt', 'w')
for item in Tst_MWF:
    thefile.write("%s\n" % item)
thefile = open('Tst_SWF.txt', 'w')
for item in Tst_SWF:
    thefile.write("%s\n" % item)

# Lengths
dev_left_length = []
tst_left_length = []
trn_left_length = []
dev_inside_length = []
tst_inside_length = []
trn_inside_length = []
dev_whole_length = []
tst_whole_length = []
trn_whole_length = []

for i in dev_map.keys():
    for j in dev_map[i]:
        if j[1] in j[0]:
            index = j[0].index(j[1])
        else:
            index = len(j[0])//2
        left = len(j[0][:index].split())
        inside = len(j[1].split())
        whole = len(j[0].split())
        dev_left_length.append(left)
        dev_inside_length.append(inside)
        dev_whole_length.append(whole)

for i in tst_map.keys():
    for j in tst_map[i]:
        if j[1] in j[0]:
            index = j[0].index(j[1])
        else:
            index = len(j[0])//2
        left = len(j[0][:index].split())
        inside = len(j[1].split())
        whole = len(j[0].split())
        tst_left_length.append(left)
        tst_inside_length.append(inside)
        tst_whole_length.append(whole)

for i in trn_map.keys():
    for j in trn_map[i]:
        if j[1] in j[0]:
            index = j[0].index(j[1])
        else:
            index = len(j[0])//2
        left = len(j[0][:index].split())
        inside = len(j[1].split())
        whole = len(j[0].split())
        trn_left_length.append(left)
        trn_inside_length.append(inside)
        trn_whole_length.append(whole)
        
# all length array concatenation
trn_length = [trn_left_length, trn_inside_length, trn_whole_length]
dev_length = [dev_left_length, dev_inside_length, dev_whole_length]
tst_length = [tst_left_length, tst_inside_length, tst_whole_length]
# all length array transform
trn_length_list=[[r[col] for r in trn_length] for col in range(len(trn_length[0]))]
dev_length_list=[[r[col] for r in dev_length] for col in range(len(dev_length[0]))]
tst_length_list=[[r[col] for r in tst_length] for col in range(len(tst_length[0]))]

from sklearn.linear_model import LogisticRegression
trn_label  = []
for i in trn_map.keys():
    for j in trn_map[i]:
        trn_label.append(j[2])

dev_label  = []
for i in dev_map.keys():
    for j in dev_map[i]:
        dev_label.append(j[2])

tst_label  = []
for i in tst_map.keys():
    for j in tst_map[i]:
        tst_label.append(j[2])

# extracted feature concatenation
trn_feature_list = [trn_mwf, float_trn_swf, trn_left_length, trn_inside_length, trn_whole_length]
dev_feature_list = [dev_mwf, dev_swf, dev_left_length, dev_inside_length, dev_whole_length]
tst_feature_list = [tst_mwf, tst_swf, tst_left_length, tst_inside_length, tst_whole_length]

# all length array transform
fixed_trn_feature_list=[[r[col] for r in trn_feature_list] for col in range(len(trn_feature_list[0]))]
fixed_dev_feature_list=[[r[col] for r in dev_feature_list] for col in range(len(dev_feature_list[0]))]
fixed_tst_feature_list=[[r[col] for r in tst_feature_list] for col in range(len(tst_feature_list[0]))]

dev_candidate_label = []
tst_candidate_label = []
for i, line in enumerate(dev_data):
    for j in range(3,len(line),3):
        dev_candidate_label.append(i)
        
for i, line in enumerate(tst_data):
    for j in range(3,len(line),3):
        tst_candidate_label.append(i)
        

clf = LogisticRegression()
clf.fit(trn_length_list, trn_label)
dev_output = clf.predict_proba(dev_length_list)
tst_output = clf.predict_proba(tst_length_list)

dev_max_output = []
for i, line in enumerate (dev_output):
    dev_max_output.append(max(line))

tst_max_output = []
for i, line in enumerate (tst_output):
    tst_max_output.append(max(line))

dev_result = [dev_candidate_label, dev_max_output]
tst_result = [tst_candidate_label, tst_max_output]

def get_dic_from_two_lists(keys, values):
    return { keys[i] : values[i] for i in range(len(keys)) }

fixed_dev_result = [[r[col] for r in dev_result] for col in range(len(dev_result[0]))]
fixed_tst_result = [[r[col] for r in tst_result] for col in range(len(tst_result[0]))]

dev_temp_max = 0
tst_temp_max = 0
dev_max_candidate_index = 0
tst_max_candidate_index = 0
dev_que_num = 0
tst_que_num = 0
dev_count = 0
tst_count = 0
dev_max_candidate_index_list = []
tst_max_candidate_index_list = []

for i, line in enumerate (fixed_dev_result):
    if line[0] == dev_que_num:
        if line[1] >= dev_temp_max:
            dev_temp_max = line[1]
            dev_max_candidate_index = dev_count
        dev_count += 1
    else: 
        dev_que_num += 1
        dev_temp_max = 0
        dev_count = 0
        dev_max_candidate_index_list.append(dev_max_candidate_index)

for i, line in enumerate (fixed_tst_result):
    if line[0] == tst_que_num:
        if line[1] >= tst_temp_max:
            tst_temp_max = line[1]
            tst_max_candidate_index = tst_count
        tst_count += 1
    else: 
        tst_que_num += 1
        tst_temp_max = 0
        tst_count = 0
        tst_max_candidate_index_list.append(tst_max_candidate_index)        

dev_answer_list = []
tst_answer_list = []

for i in range (len(dev_map) - 1):
    line = dev_map[i]
    index = dev_max_candidate_index_list[i]
    dev_answer_list.append(line[index][1])

for i in range (len(tst_map) - 1):
    line = tst_map[i]
    index = tst_max_candidate_index_list[i]
    tst_answer_list.append(line[index][1])

dev_answer_list.append('NFL')
tst_answer_list.append('NFL')

dev_pred_dict = get_dic_from_two_lists(dev_que_id, dev_answer_list)
tst_pred_dict = get_dic_from_two_lists(tst_que_id, tst_answer_list)
with open('dev_prediction_v3.json', 'w') as f:
    json.dump(dev_pred_dict, f)
with open('tst_prediction_v3.json', 'w') as f:
    json.dump(tst_pred_dict, f)